<a href="https://colab.research.google.com/github/melvingiovanniperez/machinelearningprojects/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [30]:
#Getting data from Kaggle
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle kernels list — user mepe7183 — sort-by dateRun

!kaggle competitions download -c nn-assignment

!ls

Saving kaggle.json to kaggle (1).json
kaggle.json
usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: — user mepe7183 — sort-by dateRun
testset.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
trainset.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
samplesubmission.csv: Skipping, found more recently modified local copy (use --force to force download)
'kaggle (1).json'   sample_data		   testset.csv	     trainset.csv
 kaggle.json	    samplesubmission.csv   testset.csv.zip   trainset.csv.zip


In [31]:
!unzip \*.zip
!ls

Archive:  testset.csv.zip
replace testset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: testset.csv             

Archive:  trainset.csv.zip
replace trainset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: trainset.csv            

2 archives were successfully processed.
'kaggle (1).json'   sample_data		   testset.csv	     trainset.csv
 kaggle.json	    samplesubmission.csv   testset.csv.zip   trainset.csv.zip


In [32]:
#partitioning the data
evaluate = pd.read_csv('testset.csv')
data = pd.read_csv('trainset.csv')

In [33]:
#Reviewing how data is formatted 
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,255,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
#Defining the variables
y = data['label']
X = data.drop('label', axis=1)

In [35]:
#Splitting into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [70]:
#Preparing Xs into correct format
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

#Diving by 255 to get pixel values from 0 to 1
X_train /= 255
X_test /= 255

In [37]:
#Model creation
model = keras.models.Sequential([
                                 keras.layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=[28, 28, 1]),
                                 keras.layers.MaxPooling2D(2),
                                 keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
                                 keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
                                 keras.layers.MaxPooling2D(2),
                                 keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
                                 keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
                                 keras.layers.MaxPooling2D(2),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(128, activation='relu'),
                                 keras.layers.Dropout(0.5),
                                 keras.layers.Dense(64, activation='relu'),
                                 keras.layers.Dropout(0.5),
                                 keras.layers.Dense(10, activation='softmax')])

model.summary()

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 64)        3200      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 256)        

In [38]:
#In order to reshape the Xs, they must be converted from dataframes into arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [71]:
#Reshapping so that Xs have 4 dimensions instead of 2
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [40]:
#Training the model
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
700/700 [==============================] - 317s 453ms/step - loss: 1.2978 - accuracy: 0.5301 - val_loss: 0.1824 - val_accuracy: 0.9504
Epoch 2/30
700/700 [==============================] - 316s 452ms/step - loss: 0.3046 - accuracy: 0.9182 - val_loss: 0.1236 - val_accuracy: 0.9663
Epoch 3/30
700/700 [==============================] - 315s 450ms/step - loss: 0.2077 - accuracy: 0.9458 - val_loss: 0.0980 - val_accuracy: 0.9746
Epoch 4/30
700/700 [==============================] - 315s 450ms/step - loss: 0.1616 - accuracy: 0.9582 - val_loss: 0.1035 - val_accuracy: 0.9746
Epoch 5/30
700/700 [==============================] - 315s 449ms/step - loss: 0.1265 - accuracy: 0.9671 - val_loss: 0.0766 - val_accuracy: 0.9802
Epoch 6/30
700/700 [==============================] - 321s 458ms/step - loss: 0.1051 - accuracy: 0.9718 - val_loss: 0.1191 - val_accuracy: 0.9718
Epoch 7/30
700/700 [==============================] - 317s 452ms/step - loss: 0.0939 - accuracy: 0.9755 - val_loss: 0.0980 -

In [41]:
#Testing on the X_test
score = model.evaluate(X_test, y_test, verbose=0)

In [42]:
#Accuracy of .9844 from test
print(score)

[0.10632571578025818, 0.9844642877578735]


In [43]:
#Seeing how the evaluate file is formatted
print(evaluate.head())

   pixel0  pixel1  pixel2  pixel3  ...  pixel780  pixel781  pixel782  pixel783
0       0       0       0       0  ...       255       255       255       255
1       0       0       0       0  ...         0         0         0         0
2       0       0       0       0  ...         0         0         0         0
3       0       0       0       0  ...         0         0         0         0
4       0       0       0       0  ...         0         0         0         0

[5 rows x 784 columns]


In [45]:
#Turning it into a dataframe
eval_X = evaluate[:]

       pixel0  pixel1  pixel2  pixel3  ...  pixel780  pixel781  pixel782  pixel783
0           0       0       0       0  ...       255       255       255       255
1           0       0       0       0  ...         0         0         0         0
2           0       0       0       0  ...         0         0         0         0
3           0       0       0       0  ...         0         0         0         0
4           0       0       0       0  ...         0         0         0         0
...       ...     ...     ...     ...  ...       ...       ...       ...       ...
13995       0       0       0       0  ...         0         0         0         0
13996       0       0       0       0  ...         0         0         0         0
13997       0       0       0       0  ...         0         0         0         0
13998       0       0       0       0  ...         0         0         0         0
13999       0       0       0       0  ...         0         0         0         0

[14

In [46]:
#Preparing eval_X
eval_X = eval_X.astype('float32')
eval_X /= 255

In [47]:
#Turning it into an array
eval_X = eval_X.to_numpy()

In [48]:
#reshaping 
eval_X = eval_X.reshape(eval_X.shape[0], 28, 28, 1)

In [49]:
#running the model on eval_X to receive the predictions
predictions = model.predict_classes(eval_X)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [72]:
#Verifying it went well
print(predictions.size)


14000


In [73]:
#Creating an array for ImageId to make it easier to make a new dataframe with the correct column names
ImageId = np.array(range(1,14001))

In [67]:
#Making the dataframe
testdf = pd.DataFrame({'ImageId':ImageId, 'Label':predictions})
print(testdf)

       ImageId  Label
0            1      3
1            2      1
2            3      3
3            4      7
4            5      0
...        ...    ...
13995    13996      0
13996    13997      1
13997    13998      7
13998    13999      6
13999    14000      9

[14000 rows x 2 columns]


In [68]:
#making the csv file
submission = testdf.to_csv('submission.csv', index=False)

In [69]:
#downloading the csv file
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>